 # 1️⃣ Introduction

In [ ]:
# Rakuten Benchmark Analysis - Image Model (ResNet50)
## Introduction
Ce notebook analyse le modèle utilisé par le benchmark Rakuten pour la classification d’images.

Le modèle est basé sur **ResNet50 pré-entraîné sur ImageNet** avec **27 couches décongelées**, incluant **8 couches convolutionnelles**.

L’objectif est de :
- **Comprendre l’architecture du benchmark**.
- **Reproduire le modèle utilisé**.
- **Évaluer ses performances** sur le dataset Rakuten.
- **Déterminer des axes d’amélioration** pour surpasser ce modèle.


 # 2️⃣Import des Bibliothèques

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import os


 # 3️⃣ Présentation du Modèle Image du Benchmark

In [ ]:
## Modèle utilisé dans le benchmark Rakuten
Le modèle d’image utilisé dans le benchmark est **ResNet50 pré-entraîné sur ImageNet**.  
**Modifications apportées :**
- **27 couches supérieures décongelées**.
- **8 couches convolutionnelles entraînées**.
- **Entraînement effectué avec un jeu de données non précisé (batch size, learning rate inconnus)**.

**Nombre de paramètres :**
- **12,144,667 paramètres entraînables**.
- **23,643,035 paramètres non entraînables**.


# 3️⃣ Chargement et Prétraitement des Donnée

In [ ]:
# Définition des paramètres
IMG_SIZE = (224, 224)  # Taille probable utilisée dans le benchmark
BATCH_SIZE = 64
DATA_DIR = "path_to_images"  # Remplace par ton chemin d'accès

# Prétraitement des images avec normalisation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    os.path.join(DATA_DIR, "image_train"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

val_generator = datagen.flow_from_directory(
    os.path.join(DATA_DIR, "image_train"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)


# 4️⃣ Chargement et Prétraitement des Données

In [ ]:
# Définition des paramètres
IMG_SIZE = (224, 224)  # Taille probable utilisée dans le benchmark
BATCH_SIZE = 64
DATA_DIR = "path_to_images"  # Remplace par ton chemin d'accès

# Prétraitement des images avec normalisation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    os.path.join(DATA_DIR, "image_train"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

val_generator = datagen.flow_from_directory(
    os.path.join(DATA_DIR, "image_train"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)


# 5️⃣ Reproduction du Modèle ResNet50 Benchmark

In [ ]:
# Chargement du modèle pré-entraîné ResNet50
base_model = keras.applications.ResNet50(weights="imagenet", include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

# Décongélation des 27 dernières couches
for layer in base_model.layers[:-27]:
    layer.trainable = False

# Ajout des couches finales pour la classification
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(len(train_generator.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

# Compilation du modèle
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model.summary()


 # 6️⃣ Évaluation du Modèle Benchmark

In [ ]:
# Entraînement du modèle (facultatif si modèle déjà entraîné)
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Courbes d'apprentissage
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title("Courbe d'Apprentissage - Benchmark Rakuten (ResNet50)")
plt.show()


# 7️⃣ Conclusion & Pistes d'Amélioration

In [ ]:
## Résumé des performances du benchmark :
- **Weighted-F1 score : 0.5534**
- **Points forts :** Utilisation de ResNet50, fine-tuning de 27 couches.
- **Limites :** Résultats relativement faibles, manque d’optimisation.

## Stratégie pour améliorer le benchmark :
1️⃣ **Tester d'autres modèles CNN pré-entraînés** (InceptionResNetV2, Xception, DenseNet121).  
2️⃣ **Optimiser les hyperparamètres** (batch size, learning rate, nombre de couches à fine-tuner).  
3️⃣ **Appliquer des techniques avancées d'augmentation de données** (rotation, shear, contrast, etc.).  
4️⃣ **Fusionner les prédictions texte + image** pour obtenir un modèle multimodal performant.  
